# Data Cleaning and Preparation

## 1. Handling Missing Data

In [1]:
import pandas as pd
import numpy as np

Pandas use **NaN**(Not A Number) to represent missing data, which we call a sentinel value

### 1.1 Filtering Out Missing Data

>dropna

Series:  
Return with non-null data and index

In [2]:
data = pd.Series([1, np.nan, 3])

In [3]:
data.dropna()
#data[data.notnull()]

0    1.0
2    3.0
dtype: float64

DataFrame:  
**dropna** by default drops any row containing a missing value.

In [17]:
data = pd.DataFrame([[1,2,3], [1,np.nan,np.nan], [np.nan,np.nan,np.nan], [np.nan,6,5]])

In [5]:
cleaned = data.dropna()

In [6]:
cleaned

,0,1,2
0,1.0,2.0,3.0


>dropna(how='all')

This will only drop rows that are **all NA**

In [7]:
data.dropna(how='all')

,0,1,2
0,1.0,2.0,3.0
1,1.0,NaN,NaN
3,NaN,6.0,5.0


>dropna(axis=1)

Drop columns

In [8]:
data.dropna(axis=1)
#in this case, all the columns. oops

""
0
1
2
3


>dropna(thresh=2)

Keep only rows containing **a certain number** of observations, useful in **time series data**

In [9]:
data.dropna(thresh=2)
#keep rows that at least contain 2 observations

,0,1,2
0,1.0,2.0,3.0
3,NaN,6.0,5.0


### 1.2 Filing In Missing Data

>fillna

Calling fillna with a constant replaces missing values with that value

In [10]:
data.fillna(0)

,0,1,2
0,1.0,2.0,3.0
1,1.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,6.0,5.0


In [12]:
data.fillna({1:0.5, 2:-0.5})
#Using a dict to fill different value for each column
#0 is not in the dict, so the NaNs did not fill

,0,1,2
0,1.0,2.0,3.0
1,1.0,0.5,-0.5
2,NaN,0.5,-0.5
3,NaN,6.0,5.0


>fillna(method='ffill')

Forward interpolation

In [19]:
data.fillna(method='ffill')
#just like reindex

,0,1,2
0,1.0,2.0,3.0
1,1.0,2.0,3.0
2,1.0,2.0,3.0
3,1.0,6.0,5.0


>fillna(0, inplace=True)

Modify the exising object **in-place**

In [13]:
_ = data.fillna(0, inplace=True)
data

,0,1,2
0,1.0,2.0,3.0
1,1.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,6.0,5.0


With fillna u can do lots of other things, like passing mean or median value of a Series:

In [15]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 2. Data Transformation

### 2.1 Removing Duplicates

>duplicated

Returns a boolean Series  indicating whether each row is a duplcate or not

In [21]:
data = pd.DataFrame({'k1': ['one','two']* 3 + ['two'],
                     'k2': [1,1,2,3,3,4,4]})

In [24]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

>drop_duplicates

Returns a DataFrame where the Duplicated array is *False*

In [25]:
data.drop_duplicates()
#By dufault consider all of the columns

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [32]:
data.drop_duplicates(['k1'])
#Specify any subset of them to detect duplicates

,k1,k2
0,one,1
1,two,1


### 2.1 Transforming Data Using a Function or Mapping

>map

Perform transformation based on the values in an array, Series, or columns in a DataFrame

In [34]:
data = pd.DataFrame({'food': ['bacon', 'pork', 'corned beef', 'nova lox'],
                     'ounces': [4,3,12,7]})

Mapping:  
Using a **dict-like** object:

In [39]:
meat_to_animal = {'bacon': 'pig',
                  'pork': 'pig',
                  'corned beef': 'cow',
                  'nova lox': 'salmon'}

In [40]:
data['animal'] = data['food'].map(meat_to_animal)

In [41]:
data

,food,ounces,animal
0,bacon,4,pig
1,pork,3,pig
2,corned beef,12,cow
3,nova lox,7,salmon


Using a **function**:

In [42]:
data['animal'] = data['food'].map(lambda x: meat_to_animal[x])
data

,food,ounces,animal
0,bacon,4,pig
1,pork,3,pig
2,corned beef,12,cow
3,nova lox,7,salmon


### 2.3 Replacing Values

>replace

A simple and flexible way to modify a subset of values in an object

In [47]:
data  = pd.Series([1, -999, 2, -999, -1000, 3])

In [49]:
data.replace(-999, np.nan)
#return a new Series unless use inplace=True

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

Replace multiple values, pass a **list**:

In [50]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

Different replacement for each value, pass a **list of substitutes**:

In [51]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 2.4 Renaming Axis Indexes

In [53]:
data = pd.DataFrame(np.arange(12).reshape((3,4)), 
                    index = ['Ohio', 'Colorado', 'New York'],
                    columns = ['one', 'two', 'three', 'four'])

>map

Like a Series, the axis indexes have a map method:

In [54]:
transform = lambda x: x[:4].upper()

In [55]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

>rename

Can be used in conjunction with **dict-like** object for a subset of the axis labels

In [62]:
data.rename(index=str.lower, columns=str.upper)
#pass inplace=True to modify inplace

,ONE,TWO,THREE,FOUR
ohio,0,1,2,3
colorado,4,5,6,7
new york,8,9,10,11


In [63]:
data.rename(index={'Ohio': 'Indiana'})

,one,two,three,four
Indiana,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


### 2.5 Discretization and Binning

**Continuous** data is often discretized or otherwise separated into "bins" for analysis

>**cut**  function in pandaas

Return a special **Categorical object**  
When passing bin edges:

In [64]:
ages = [20, 22, 25, 27 ,21, 23 ,37, 31, 61, 45, 41, 32]

In [72]:
#bin edges
bins = [18, 25,35, 60,100]

In [71]:
cats = pd.cut(ages, bins)
#U can pass ur own bin names by passing a list or array to the labels option, like labels=['Youth', 'YoungAdult',...]
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [68]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [69]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [70]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

When passing **an interger number**, it will compute equal-length bins based on the **minimun and maximun values**:

In [80]:
data = np.random.rand(20)

In [81]:
pd.cut(data, 4, precision=2)
#Presicion option limits the decimal precision to two digits

[(0.27, 0.5], (0.27, 0.5], (0.27, 0.5], (0.74, 0.98], (0.74, 0.98], ..., (0.74, 0.98], (0.029, 0.27], (0.029, 0.27], (0.029, 0.27], (0.029, 0.27]]
Length: 20
Categories (4, interval[float64]): [(0.029, 0.27] < (0.27, 0.5] < (0.5, 0.74] < (0.74, 0.98]]

>**qcut** function in pandas

Return roufhly **equal-size**(quantity) bins

In [87]:
data = np.random.rand(100)

In [88]:
cats = pd.qcut(data, 4)
cats

[(0.561, 0.817], (0.817, 0.982], (0.817, 0.982], (0.00514, 0.334], (0.817, 0.982], ..., (0.817, 0.982], (0.00514, 0.334], (0.817, 0.982], (0.817, 0.982], (0.561, 0.817]]
Length: 100
Categories (4, interval[float64]): [(0.00514, 0.334] < (0.334, 0.561] < (0.561, 0.817] < (0.817, 0.982]]

In [90]:
pd.value_counts(cats)

(0.817, 0.982]      25
(0.561, 0.817]      25
(0.334, 0.561]      25
(0.00514, 0.334]    25
dtype: int64

### 2.6 Detecting Filtering Outliers

In [99]:
data = pd.DataFrame(np.random.randn(100,4))

In [100]:
data.describe()

,0,1,2,3
count,100.000000,100.000000,100.000000,100.000000
mean,0.184456,0.071841,0.164331,0.093503
std,0.985147,1.014924,1.041199,1.012658
min,-2.421873,-2.559582,-3.280173,-3.607572
25%,-0.382153,-0.447871,-0.475128,-0.446102
50%,0.117520,0.175920,0.196891,0.249913
75%,0.767847,0.623083,0.880797,0.722499
max,2.943548,2.248116,2.440341,2.086467


>**any** method on a boolean DataFrame

Select rows that have **one** value ... :  

In [101]:
data[(np.abs(data)>3).any(1)]

,0,1,2,3
59,-1.056467,0.612886,-3.280173,0.098085
82,0.474763,-0.954024,-0.314368,-3.607572


>**np.sign(data)**

Produces 1 and -1 values based on if its positive or negative

In [102]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,1.0,-1.0
2,1.0,-1.0,-1.0,-1.0
3,1.0,1.0,1.0,1.0
4,1.0,-1.0,1.0,-1.0


**Cap values** outside the interval -3 to 3

In [103]:
data[np.abs(data)>3] = np.sign(data)*3

In [105]:
data.describe()

,0,1,2,3
count,100.000000,100.000000,100.000000,100.000000
mean,0.184456,0.071841,0.167133,0.099578
std,0.985147,1.014924,1.032175,0.991836
min,-2.421873,-2.559582,-3.000000,-3.000000
25%,-0.382153,-0.447871,-0.475128,-0.446102
50%,0.117520,0.175920,0.196891,0.249913
75%,0.767847,0.623083,0.880797,0.722499
max,2.943548,2.248116,2.440341,2.086467


### 2.7 Permutation and Random Sampling

>**np.random.permutation** function

Produces an array of integers indicating the new ordering

In [106]:
df = pd.DataFrame(np.arange(5*4).reshape(5,4))

In [110]:
#bc df has 5 rows:
sampler = np.random.permutation(5)
sampler

array([3, 1, 0, 2, 4])

>**take** function

Equivalent of **iloc-based indexing**

In [111]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19


>**sample** method on Series and DataFrame

Select a random subset

In [113]:
df.sample(n=3)
#replace =  True will allow repeat choices

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
0,0,1,2,3


### 2.8 Computing Indicator / Dummy Variables

>**get_dummies** function in pandas

**dummy veriable** is one that takes only the value 0 or 1 to indicate the absence or presence of some categorical effect that may be expected tosift the outcome.

In [116]:
df = pd.DataFrame({'key': ['b','b','a','c','a','b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [117]:
pd.get_dummies(df.key)

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


>pd.get_dummies(data, **prefix='...'**)

Add a **prefix** to the columns in the indicator DataFrame

In [118]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [124]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


Combine **get_dummies** with a discretization function like **cut**:

In [125]:
np.random.seed(12345)

In [126]:
values = np.random.rand(10)

In [128]:
bins =[0,0.2,0.4,0.6,0.8,1]

In [129]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0
